In [1]:
import pandas as pd
import os
import io
from pathlib import Path
import numpy as np
import re


In [2]:
from selenium import webdriver

from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import NoSuchElementException

In [3]:
all = pd.read_excel('Data/Analysis of Turizm Journals.xlsx', sheet_name=None, usecols="A:D", header=1 )

In [4]:
all.keys()

dict_keys(['General Information', 'IJCHM', 'IJHM', 'CHQ', 'JHTR', 'JHMM'])

In [5]:
del all["General Information"]

In [6]:
df = pd.concat(all)

In [7]:
df

AU  \
IJCHM 0            Ineson, EM; Comic, D; Kalmic, L   
      1                        Benitez-Aurioles, B   
      2                   Hu, XB; Yang, Y; Park, S   
      3        Ahmat, NHC; Arendt, SW; Russell, DW   
      4                        Kim, EL; Tanford, S   
...                                            ...   
JHMM  224           de Leaniz, PMG; Rodriguez, IRD   
      225  Prayag, G; Khoo-Lattimore, C; Sitruk, J   
      226                               Morosan, C   
      227                 Memarzadeh, F; Chang, HJ   
      228                           Cao, Y; Kim, K   

                                                          TI      PY  \
IJCHM 0    Towards a hierarchy of hotel guests' in-room n...  2019.0   
      1    Barcelona's peer-to-peer tourist accommodation...  2019.0   
      2    A meta-regression on the effect of online rati...  2019.0   
      3    Examining work factors after Malaysia's minimu...  2019.0   
      4    Seeking reward or avoiding risk from restauran...  2019.0   
...                                                      ...     ...   
JHMM  224  Exploring the Antecedents of Hotel Customer Lo...  2015.0   
      225  Casual Dining on the French Riviera: Examining...  2015.0   
      226  The Influence of DMO Advertising on Specific D...  2015.0   
      227  Online Consumer Complaints About Southeast Asi...  2015.0   
      228  How Do Customers Perceive Service Quality in D...  2015.0   

                                     DI  
IJCHM 0      10.1108/IJCHM-01-2019-0061  
      1      10.1108/IJCHM-01-2019-0090  
      2      10.1108/IJCHM-10-2018-0835  
      3      10.1108/IJCHM-10-2018-0827  
      4      10.1108/IJCHM-03-2018-0235  
...                                 ...  
JHMM  224  10.1080/19368623.2014.891961  
      225  10.1080/19368623.2014.859114  
      226  10.1080/19368623.2014.891962  
      227  10.1080/19368623.2014.893222  
      228  10.1080/19368623.2014.903817  

[3269 rows x 4 columns]

In [8]:
df.isnull().sum()

AU     0
TI     0
PY    19
DI     1
dtype: int64

In [9]:
df.reset_index(level=0).groupby("level_0").count()

,AU,TI,PY,DI
level_0,,,,
CHQ,356,356,353,356
IJCHM,1008,1008,1008,1007
IJHM,1340,1340,1340,1340
JHMM,229,229,217,229
JHTR,336,336,332,336


In [10]:
df = df.reset_index(level=0)

In [11]:
df.rename(columns={'level_0': 'JO'}, inplace=True)

In [12]:
df.reset_index(drop=True, inplace=True)

In [13]:
df

,JO,AU,TI,PY,DI
0,IJCHM,"Ineson, EM; Comic, D; Kalmic, L",Towards a hierarchy of hotel guests' in-room n...,2019.0,10.1108/IJCHM-01-2019-0061
1,IJCHM,"Benitez-Aurioles, B",Barcelona's peer-to-peer tourist accommodation...,2019.0,10.1108/IJCHM-01-2019-0090
2,IJCHM,"Hu, XB; Yang, Y; Park, S",A meta-regression on the effect of online rati...,2019.0,10.1108/IJCHM-10-2018-0835
3,IJCHM,"Ahmat, NHC; Arendt, SW; Russell, DW",Examining work factors after Malaysia's minimu...,2019.0,10.1108/IJCHM-10-2018-0827
4,IJCHM,"Kim, EL; Tanford, S",Seeking reward or avoiding risk from restauran...,2019.0,10.1108/IJCHM-03-2018-0235
...,...,...,...,...,...
3264,JHMM,"de Leaniz, PMG; Rodriguez, IRD",Exploring the Antecedents of Hotel Customer Lo...,2015.0,10.1080/19368623.2014.891961
3265,JHMM,"Prayag, G; Khoo-Lattimore, C; Sitruk, J",Casual Dining on the French Riviera: Examining...,2015.0,10.1080/19368623.2014.859114
3266,JHMM,"Morosan, C",The Influence of DMO Advertising on Specific D...,2015.0,10.1080/19368623.2014.891962
3267,JHMM,"Memarzadeh, F; Chang, HJ",Online Consumer Complaints About Southeast Asi...,2015.0,10.1080/19368623.2014.893222


In [14]:
example = df[df["JO"]=="IJHM"].iloc[0] 
example

JO                                                 IJHM
AU                                   Yang, FX; Lau, VMC
TI    Evil customers, an angel boss and coopetitive ...
PY                                                 2019
DI                           10.1016/j.ijhm.2019.04.004
Name: 1008, dtype: object

In [15]:
driver = webdriver.Chrome("chromedriver.exe")

In [16]:
driver.get("https://www.sciencedirect.com/science/article/pii/S0278431919300258")

In [17]:
driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="gh-main-cnt"]/div[1]/div/nav[2]/ul/li[1]/a'))))

In [18]:
driver.find_element_by_xpath('//*[@id="search-input"]').send_keys(example["DI"])       

In [19]:
driver.find_element_by_xpath('//*[@id="search-input"]').send_keys(Keys.RETURN)       

In [20]:
first_link = driver.find_element_by_xpath('//*[@id="srp-results-list"]/ol/li[1]/div/div[2]/h2/span/a').get_attribute("href")
first_link

'https://www.sciencedirect.com/science/article/pii/S0278431918309782'

In [21]:
driver.get(first_link)

In [22]:
secs =driver.find_elements_by_xpath('//div[@class="Body u-font-serif"]/div/section')

doc = "" 

i=1
for sec in secs:
    doc += f"***\nSECTION 00{str(i)}\n***"
    doc += "*****\n*****"
    doc += sec.text
    i+=1

In [ ]:
doc

In [24]:
filename="example.txt"
with io.open(f"Data/example/{filename}", "w", encoding="utf-8") as text_file:
    text_file.write(doc)

In [104]:
df_all = pd.read_excel("Data/paper_final_30Jan.xlsx", usecols="B:K")
df_all

,JO,AU,TI,PY,DI,urls,filename
0,IJCHM,"Ineson, EM; Comic, D; Kalmic, L",Towards a hierarchy of hotel guests' in-room n...,2019.0,10.1108/IJCHM-01-2019-0061,https://www.emerald.com/insight/content/doi/10...,IJCHM_0.txt
1,IJCHM,"Benitez-Aurioles, B",Barcelona's peer-to-peer tourist accommodation...,2019.0,10.1108/IJCHM-01-2019-0090,https://www.emerald.com/insight/content/doi/10...,IJCHM_1.txt
2,IJCHM,"Hu, XB; Yang, Y; Park, S",A meta-regression on the effect of online rati...,2019.0,10.1108/IJCHM-10-2018-0835,https://www.emerald.com/insight/content/doi/10...,IJCHM_2.txt
3,IJCHM,"Ahmat, NHC; Arendt, SW; Russell, DW",Examining work factors after Malaysia's minimu...,2019.0,10.1108/IJCHM-10-2018-0827,https://www.emerald.com/insight/content/doi/10...,IJCHM_3.txt
4,IJCHM,"Kim, EL; Tanford, S",Seeking reward or avoiding risk from restauran...,2019.0,10.1108/IJCHM-03-2018-0235,https://www.emerald.com/insight/content/doi/10...,IJCHM_4.txt
...,...,...,...,...,...,...,...
3264,JHTR,"Kim, W; Ok, C",CUSTOMER ORIENTATION OF SERVICE EMPLOYEES AND ...,2010.0,10.1177/1096348009344234,NaN,NaN
3265,JHTR,"Gazzoli, G; Hancer, M; Park, Y",THE ROLE AND EFFECT OF JOB SATISFACTION AND EM...,2010.0,10.1177/1096348009344235,NaN,NaN
3266,JHTR,"Young, CA; Corsun, DL","BURNED! THE IMPACT OF WORK ASPECTS, INJURY, AN...",2010.0,10.1177/1096348009349816,NaN,NaN
3267,JHTR,"Kim, DY; Wen, L; Doh, K",DOES CULTURAL DIFFERENCE AFFECT CUSTOMER'S RES...,2010.0,10.1177/1096348009349817,NaN,NaN


In [105]:
df.filename.isnull().sum()

114